# Navigation

Splitting the dataset into above and below median for several personality traits and building BoW models seperately on each of them.

In [16]:
import pandas as pd
import os as os
import json as js
import gzip
import numpy as np
import matplotlib.pyplot as plt
import pickle

#importing libraries for NLP
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter

#Text Classification Material
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix

from sklearn import  linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MAHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Analysis

In [8]:
with open ('df_07_08', 'rb') as fr:
    df = pickle.load(fr)

In [12]:
df.columns[12:67]
#len(df.columns)

Index(['length', '!', '?', ',', '.', ' ', 'Extro', 'Agree', 'Neuro', 'Open',
       'Consc', 'Extro_Comedy', 'Extro_Art House & International',
       'Extro_Drama', 'Extro_Action & Adventure', 'Extro_Horror',
       'Extro_Science Fiction', 'Extro_Animation', 'Agree_Comedy',
       'Agree_Art House & International', 'Agree_Drama',
       'Agree_Action & Adventure', 'Agree_Horror', 'Agree_Science Fiction',
       'Agree_Animation', 'Neuro_Comedy', 'Neuro_Art House & International',
       'Neuro_Drama', 'Neuro_Action & Adventure', 'Neuro_Horror',
       'Neuro_Science Fiction', 'Neuro_Animation', 'Open_Comedy',
       'Open_Art House & International', 'Open_Drama',
       'Open_Action & Adventure', 'Open_Horror', 'Open_Science Fiction',
       'Open_Animation', 'Consc_Comedy', 'Consc_Art House & International',
       'Consc_Drama', 'Consc_Action & Adventure', 'Consc_Horror',
       'Consc_Science Fiction', 'Consc_Animation', 'corpus', 'noun', 'adj',
       'verb (simple)', 'verb (simp

## Linear Regression

In [17]:
X = df.drop(columns=["corpus",'Extro_Comedy', 'Extro_Art House & International',
       'Extro_Drama', 'Extro_Action & Adventure', 'Extro_Horror',
       'Extro_Science Fiction', 'Extro_Animation', 'Agree_Comedy',
       'Agree_Art House & International', 'Agree_Drama',
       'Agree_Action & Adventure', 'Agree_Horror', 'Agree_Science Fiction',
       'Agree_Animation', 'Neuro_Comedy', 'Neuro_Art House & International',
       'Neuro_Drama', 'Neuro_Action & Adventure', 'Neuro_Horror',
       'Neuro_Science Fiction', 'Neuro_Animation', 'Open_Comedy',
       'Open_Art House & International', 'Open_Drama',
       'Open_Action & Adventure', 'Open_Horror', 'Open_Science Fiction',
       'Open_Animation', 'Consc_Comedy', 'Consc_Art House & International',
       'Consc_Drama', 'Consc_Action & Adventure', 'Consc_Horror',
       'Consc_Science Fiction', 'Consc_Animation',"sentiment"])
X1 = X.iloc[:,12:66]
y = df["good"]

X2 = sm.add_constant(X1)
est = sm.OLS(y, X2).fit()
print(est.summary())

#result = pd.concat((est.params, est.pvalues), axis=1)
#result.rename(columns={0: 'beta', 1: 't'}).to_excel('linear_regression.xls', 'sheet1')

# FULL: adjusted r squaed 0.133
# Without interaction terms: 0.125

                            OLS Regression Results                            
Dep. Variable:                   good   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.125
Method:                 Least Squares   F-statistic:                     142.2
Date:                Sat, 10 Aug 2019   Prob (F-statistic):               0.00
Time:                        12:22:31   Log-Likelihood:                -12388.
No. Observations:               18810   AIC:                         2.482e+04
Df Residuals:                   18790   BIC:                         2.497e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0

C:\Users\MAHAM\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## Hierarchical Model

In [2]:
with open ('df_07_08', 'rb') as fr:
    df = pickle.load(fr)

In [6]:
df.shape

(18810, 68)

In [5]:
with open ('data_26_07_w_corpus', 'rb') as fr:
    df2 = pickle.load(fr)
df2.shape

(18810, 59)

In [7]:
median_O = df.loc[:,"Open"].median()
median_N = df.loc[:,"Neuro"].median()
median_A = df.loc[:,"Agree"].median()

# dataframe with high openness
df_H_O = df[df.Open >= median_O]
# df with low openness
df_L_O = df[df.Open < median_O]
# dataframe with high openness
df_H_N = df[df.Neuro >= median_N]
# df with low openness
df_L_N = df[df.Neuro < median_N]

# dataframe with high Agreeableness
df_H_A = df[df.Agree >= median_A]
# df with low Agreeableness
df_L_A = df[df.Agree < median_A]

In [8]:
median_A, median_N, median_O

(-0.21503994493254414, 0.3723573823917685, -0.13265988405009393)

In [9]:
import math
# df with random split
# reshuffle first
df = df.sample(frac=1).reset_index(drop=True)
half = math.ceil(len(df)/2)
df1 = df.iloc[:half,:]
df2 = df.iloc[half:,]

In [10]:
dataframes = [df1,df2,df_H_O,df_L_O,df_H_N,df_L_N,df_H_A,df_L_A]
dataframes_names = ["df1","df2","df_H_O","df_L_O","df_H_N","df_L_N","df_H_A","df_L_A"]
for dfx, name in zip(dataframes,dataframes_names):
    print (name, dfx.shape)

df1 (9405, 68)
df2 (9405, 68)
df_H_O (9405, 68)
df_L_O (9405, 68)
df_H_N (12248, 68)
df_L_N (6562, 68)
df_H_A (14964, 68)
df_L_A (3846, 68)


# RELEVANT PART

In [12]:
# df1 and df2 are random splits and df_H_O means High Openness, 
# df_L_O means low opennes etc, I split my median
dataframes = [df1,df2,df_H_O,df_L_O,df_H_N,df_L_N,df_H_A,df_L_A]
dataframes_names = ["df1","df2","df_H_O","df_L_O","df_H_N","df_L_N","df_H_A","df_L_A"]

In [13]:
for dfx, name in zip(dataframes,dataframes_names):
    cv = CountVectorizer(max_features = 500, ngram_range=(1,2))
    XX = cv.fit_transform(dfx["corpus"]).toarray()
    # using relative word counts instead of absolute ones
    tfidf_transformer = TfidfTransformer()
    tfidf = tfidf_transformer.fit_transform(XX).toarray()
    X = tfidf
    y=dfx["good"]
    acc=[]
    for seed in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = seed)
        log = LogisticRegression(solver='liblinear')#regularization is applied by default
        log.fit(X_train, y_train)
        accuracy=log.score(X_test,y_test)
        acc.append(accuracy)
    print(name," ",sum(acc)/len(acc), len(dfx))

df1   0.8186366213151925 9405
df2   0.8144274376417233 9405
df_H_O   0.813421201814059 9405
df_L_O   0.818792517006803 9405
df_H_N   0.8175919878075331 12248
df_L_N   0.8214503351614868 6562
df_H_A   0.8185957408892452 14964
df_L_A   0.7974012474012474 3846


# END

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
#X = df.iloc[:,5:]
#y = df["good"]
clf = RidgeClassifierCV(alphas=[1e-7,1e-6,1e-5,1e-4,1e-3, 1e-2, 1e-1, 1]).fit(X_train, y_train)
clf.score(X_test, y_test) 

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
#X = df.iloc[:,5:]
#y = df["good"]
mlp = MLPClassifier(alpha=0.9,random_state=123)
mlp.fit(X_train, y_train)
score = mlp.score(X_test, y_test)
score